# Not what she had hoped for

dr. Elena Vasilsecu powered on the MARS-O-HELP. She had spent the last couple of [sols](https://en.wikipedia.org/wiki/Mars_sol) attaching the satellite dish to the main arm of the robot, and she was getting very enthusiastic about finally communicating with another human. It's been 2 very long months (to be more accurate, 63 sols), and she was finally starting to see the end of the tunnel.

She had calculated the trajectory of one of the [Mars Relay Network](https://mars.nasa.gov/news/8861/the-mars-relay-network-connects-us-to-nasas-martian-explorers/) satellites, and had converted that into a pose trajectory for the robot. She had prerecorded some messages to be transmitted to the satellites and was waiting for the right moment to start executing the trajectory. While she was waiting, all kind of thoughts run through her head: if she tightened the screws properly, if the transformation from the robot to the Mars frame was correct, whether she used the right Euler angle sequence for calculating the orientation. But time was running low, and she had to focus: if she wanted to track the satellite with her robot, she had to start executing the trajectory in 3,…2,…1,… Now!

After she pressed the button, the robot started moving and was executing the trajectory! It was a bliss to watch for a couple of seconds, until suddenly the robot started wiggling, swiveling, and loosing balance…

<img src="../artwork/stranded/robot_failure.png" width=60%/>

After a loud cry, that was heard by only one person in the whole Universe, she watched the MARS-O-HELP lying on the ground trying to move around the huge satellite dish. She didn't understand, she had everything calculated! What went wrong, why couldn't the MARS-O-HELP follow the designed trajectory? What could she possibly be missing?

## Robot arm process dynamic modeling

When we want to control a process, we must know how the input to the process affects its output. We usually describe such a relationship with what we call a __dynamic model__ of the process. The relationship is often described in the form of a system of ordinary differential equations: the forces/torques at the joint level and joint states are the __input__ and the acceleration at the joint level is the __output__ of the process. By integrating such equations over time, we can calculate the joint coordinates over time.

Differential equations describing input force and output acceleration have been first described by Newton's second law of motion ($ F = m \ddot{x}$). We could try to model the robot's motion using this law, however it is generally very complicated due to the fact that reaction forces at the joint level are very difficult to calculate. We therefore resort to the more general formulation of physics developed by [Euler and Lagrange](https://en.wikipedia.org/wiki/Euler%E2%80%93Lagrange_equation).

### Euler-Lagrange formulation of physics

The Euler-Lagrange formulation defines a quantity for a given system, called the Lagrangian, defined as:

$$L(q,\dot{q})=K(q,\dot{q})-P(q) $$

where $K$ represents the total kinetic energy of the system and $P$ represents the total potential energy of the system. 
The Euler-Lagrange equations that describe the dynamics of a $n-DOF$ mechanical system are:

$$ \frac{d}{dt}\frac{\partial L}{\partial \dot{q}_i}-\frac{\partial L}{\partial q_i}=\tau_i, \qquad i=1,...,n, $$

where $q_i$ represent generalized coordinates (in our case the joint coordinates) and $\tau_i$ generalized forces (in our case motor forces/torques)

The matrix form of the Euler-Lagrange equations is:

$$ D(q)\ddot{q}+C(q,\dot{q})\dot{q}+G(q)=\tau $$

where $q= \begin{bmatrix}q_1\\ \vdots \\q_n \end{bmatrix} and \tau=\begin{bmatrix}\tau_1 \\ \vdots \tau_n \end{bmatrix}$. 

The matrix $D(q)$ is called the inertia matrix, it is symmetric and positive definite, and can be expressed in terms of the kinetic energy:

$$ K=\frac{1}{2} \dot{q}^T D(q) \dot{q}=\frac{1}{2}\sum_{i,j}^{n} d_{i,j}(q)\dot{q}_i\dot{q}_j $$

The matrix $C(q)$ takes into account centrifugal and Coriolis terms, and each $k,j-th$ matrix element can be calculated as:

$$ c_{kj}=\frac{1}{2}\sum_{i=1}^{n} 
\bigg \{
\underbrace{
\frac{\partial d_{kj}}{\partial q_i}+\frac{\partial d_{ki}}{\partial q_j}-\frac{\partial d_{ij}}{\partial q_k}
}_{c_{ijk}}
\bigg \}
\dot{q}_i.  $$

The last term $G(q)$, sometimes called gravity term, is a column vector $G=\begin{bmatrix} g_1 \\ \vdots \\ g_n \end{bmatrix}$, where each $k-th$ term is derived from the potential energy:

$$ g_k(q)=\frac{\partial P}{\partial q_k },  \qquad k=1,...,n. $$

## A 2DOF robot arm with spatial movement

Consider a 2DOF robot arm with two revolute joints, that can move in a 3D Cartesian space, with the schematic representation from the figure below. Because the first rotation axis is on the X axis, and the second on an axis perpendicular to the first, the robot can move in a 3D space.

<img src="../artwork/DynMod/2DOF.png" width=40% />

### Forward kinematics

The forward kinematics can be derived through transformation matrices from the base frame to the end effector frame. The base frame coincides with the first frame (that is the frame of joint 1, with origin $O_1$ in the center of the joint). Thus, the transformation matrix $T_{01}$ is simply a rotation around X: 

$$ T_0^1 = Rx(q_1)=\begin{bmatrix}
 1 &       0 &        0 & 0\\
 0 & cos(q_1) & -sin(q_1) & 0\\
 0 & sin(q_1) &  cos(q_1) & 0\\
 0 &       0 &        0 & 1
\end{bmatrix} $$


From Frame 1 we arrive at Frame 2 (corresponding to the joint 2) through a translation on Z and a rotation around Y ($T_1^2$):

$$ T_1^2 = Transl(z,L_1) Ry(q_2)=\begin{bmatrix}
  cos(q2) & 0 & sin(q2) &  0\\
       0 & 1 &       0 &  0\\
 -sin(q2) & 0 & cos(q2) & L_1\\
 0 &       0 &        0 & 1
\end{bmatrix} $$

Finally, the end effector frame is obtained through a translation on Z ($T_2^3$): 
 
$$ T_2^3 = Transl(z,L_2) = \begin{bmatrix}
 1 &       0 &        0 & 0\\
 0 & 1 & 0 & 0\\
 0 & 0 &  1	 & L_2\\
 0 &       0 &        0 & 1
\end{bmatrix} $$

The transformation matrix from the based frame to the end effector, that is the forward kinematics, is obtained through multiplication:

$$  T_0^3=T_0^1 T_1^2 T_2^3= \begin{bmatrix}
          cos(q_2)&       0&          sin(q_2)&                 L_2sin(q_2)\\
  sin(q_1)sin(q_2)& cos(q_1)& -cos(q_2)sin(q_1)& -sin(q_1)(L_1 + L_2cos(q_2))\\
 -cos(q_1)sin(q_2)& sin(q_1)&  cos(q_1)cos(q_2)&  cos(q_1)(L_1 + L_2cos(q_2))\\
                0&       0&                0&                          1
\end{bmatrix}$$

The position of the end effector with respect to the joint angles $q_1$ and $q_2$ is given by the first three elements of the 4th column:

$$
P_x = L_2 sin(q_1),
P_y = -L_1 sin(q_1)-L_2 sin(q_1) cos(q_2),
P_z = L_1 cos(q_1)+L_2 cos(q_1) cos(q_2)  \text{ (11)}
$$

The orientation of the end effector is given by the submatrix R (lines 1-3 and columns 1-3 of T):

$$ R = \begin{bmatrix}
          cos(q_2)&       0&          sin(q_2)\\
  sin(q_1)sin(q_2)& cos(q_1)& -cos(q_2)sin(q_1)\\
 -cos(q_1)sin(q_2)& sin(q_1)&  cos(q_1)cos(q_2)
\end{bmatrix} $$

### Jacobian

The Jacobian relates the joint velocities to the linear and angular velocities of the end effector. For the 2DOF robot arm from the figure above, with the geometric model presented, the Jacobian is:

$$ J = \begin{bmatrix}
        0                                  &       \frac{L_2}{2}cos(q_2)    \\
-\frac{L_2}{2}cos(q_1)cos(q_2)-L_1cos(q_1) &       \frac{L_2}{2}sin(q_1)sin(q_2) \\
-\frac{L_2}{2}sin(q_1)cos(q_2)-L_1sin(q_1) &       -\frac{L_2}{2}cos(q_1)sin(q_2) \\
          1                                &       0 \\       
          0                                &       cos(q_1) \\ 
          0                                &       sin(q_1) 
\end{bmatrix} $$

Thus, if we refer to link 2, the angular and linear Jacobians are:

$$ J_{vc2}=J_{vc}= \begin{bmatrix}
        0                                  &       \frac{L_2}{2}cos(q_2)    \\
-\frac{L_2}{2}cos(q_1)cos(q_2)-L_1cos(q_1) &       \frac{L_2}{2}sin(q_1)sin(q_2) \\
-\frac{L_2}{2}sin(q_1)cos(q_2)-L_1sin(q_1) &       -\frac{L_2}{2}cos(q_1)sin(q_2)
\end{bmatrix} $$

$$ J_{\omega2}=J_{\omega}=
\begin{bmatrix}
          1                                &       0 \\       
          0                                &       cos(q_1) \\ 
          0                                &       sin(q_1) 
\end{bmatrix} $$ 

Further on, the angular and linear Jacobians for link 1 can be determined as:

$$ J_{vc1} =
\begin{bmatrix}
        0              &       0    \\
-\frac{L_1}{2}cos(q_1) &       0 \\
-\frac{L_1}{2}sin(q_1)  &       0
\end{bmatrix} $$

$$ J_{\omega1}=
\begin{bmatrix}
          1                                &       0 \\       
          0                                &       0 \\ 
          0                                &       0 
\end{bmatrix} $$

### Lagrangian

The Lagrangian is composed out of kinetic energy and potential energy. The kinetic energy has a linear and an angular component

$$ K = K_{lin}+K_{ang} $$

given by the expressions:
$$ K_{lin}=\frac{1}{2}m_1 u_{c1}^T u_{c1}+\frac{1}{2}m_2 u_{c2}^T u_{c2}=\frac{1}{2} \dot{q}^T (m_1 J_{u1}^T J_{u1}+m_2 J_{u2}^T J_{u2}) \dot{q} $$

and
$$ K_{ang}=\frac{1}{2} \dot{q}^T(J_{\omega2}^T R_0^2 I_2 R_0^{2_T} J_{\omega2}+J_{\omega1}^T R_0^1 I_1 R_0^{1_T} J_{\omega1})\dot{q} $$

with 
$$ R_0^2 = R, \quad R_0^1=T_0^1(1:3,1:3), I_2=diag\{0,I_{2y},0\},\quad I_1=diag\{I_{1x},0,0\} $$

After calculating the expressions for both components of the kinetic energy, we obtain the inertia matrix $D(q)$ as

$$
D(q)=\begin{bmatrix}
          d_{11}&    d_{12}\\
          d_{21}&    d_{22}
\end{bmatrix}=\begin{bmatrix}
  I_{1x}+\frac{L_1^2 m_1}{4}+L_1^2m_2+\frac{L_2^2 m_2}{4}cos^2(q_2)+L_1L_2m_2cos(q_2)   &    0\\
     0																					&    \frac{m_2 L_2^2}{4}+I_{2y}
\end{bmatrix} $$

In deriving matrix $C(q,\dot{q})$, we first calculate each $c_{ijk}$ term from equation (5):

$$ \begin{matrix}
  c_{111}=\frac{\partial d_{11}}{\partial q_1}+\frac{\partial d_{11}}{\partial q_1}-\frac{\partial d_{11}}{\partial q_1}=0   \\
     c_{112}=\frac{\partial d_{21}}{\partial q_1}+\frac{\partial d_{21}}{\partial q_1}-\frac{\partial d_{11}}{\partial q_2}=\frac{L_2^2 m_2}{4}sin(2q_2)+L_1L_2m_2sin(q_2)																			\\
     c_{121}=\frac{\partial d_{12}}{\partial q_1}+\frac{\partial d_{11}}{\partial q_2}-\frac{\partial d_{12}}{\partial q_1}=-\frac{L_2^2 m_2}{4}sin(2q_2)-L_1L_2m_2sin(q_2) \\
     c_{122}=\frac{\partial d_{22}}{\partial q_1}+\frac{\partial d_{21}}{\partial q_2}-\frac{\partial d_{12}}{\partial q_2}=0 \\
     c_{211}=\frac{\partial d_{11}}{\partial q_2}+\frac{\partial d_{12}}{\partial q_1}-\frac{\partial d_{21}}{\partial  q_1}=c_{121} \\
     c_{212}=\frac{\partial d_{21}}{\partial q_2}+\frac{\partial d_{22}}{\partial q_1}-\frac{\partial d_{21}}{\partial q_2}=0 \\
     c_{221}=\frac{\partial d_{12}}{\partial q_2}+\frac{\partial d_{12}}{\partial q_2}-\frac{\partial d_{22}}{\partial q_1}=0 \\
     c_{222}=\frac{\partial d_{22}}{\partial q_2}+\frac{\partial d_{22}}{\partial q_2}-\frac{\partial d_{22}}{\partial q_2}=0
\end{matrix} $$

In the end we obtain the matrix:
$$ C(q,\dot{q})= \begin{bmatrix} -\frac{L_2^2 m_2}{8}sin(2q_2)\dot{q}_2-\frac{1}{2} L_1L_2m_2sin(q_2)\dot{q}_2  &  -\frac{L_2^2 m_2}{8}sin(2q_2)\dot{q}_1-\frac{1}{2}L_1L_2m_2sin(q_2)\dot{q}_1\\
         \frac{L_2^2 m_2}{8}sin(2q_2)\dot{q}_1+\frac{1}{2}L_1L_2m_2sin(q_2)\dot{q}_1  &  0
\end{bmatrix} $$

The potential energy is determined by multiplying the mass by the gravitational acceleration and the height of the center of mass:
$$ P_1=m_1g\frac{L_1}{2}cos(q_1), \text{ (24)}\\
P_2=m_2g \Big (L_1cos(q_1)+\frac{L_2}{2}cos(q_1)cos(q_2) \Big),
P=P_1+P_2. $$

The gravity term is determined as:
$$ G(q) = \begin{bmatrix}
-\frac{m_1gL_1+2m_2gL_1}{2}sin(q_1)-\frac{m_2gL_2}{2}sin(q_1)cos(q_2)\\
-\frac{m_2gL_2}{2}cos(q_1)sin(q_2)
\end{bmatrix} $$

This completes the dynamic model for our robot arm.